In [1]:
from nlp_models.gemma1.load_model import get_gemma1

lm_cache_dir = '/data1/bumjin/datahub'
model, tokenizer = get_gemma1('gemma1', '7b', lm_cache_dir, device_map='cpu')
print(model)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps=1

In [2]:
model.config

GemmaConfig {
  "_name_or_path": "google/gemma-7b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 24576,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 256000
}

In [3]:
from nlp_models.utils.find_vocab_index import find_vocab_index, find_inclusion_vocab_index, decode_token_index

find_vocab_index(tokenizer, 'yes'), find_inclusion_vocab_index(tokenizer, 'yes')

print("====================")
for index in find_vocab_index(tokenizer, 'yes'):
    if index is not None:
        print(index, decode_token_index(tokenizer, index))
print("====================")
for index in find_inclusion_vocab_index(tokenizer, 'yes'):
    print(index, decode_token_index(tokenizer, index))

3276 yes
3553 Yes
17480 YES
172060 ▁employes
173003 ▁egyes
200635 Hayes
123951 Yess
175357 YesNo
200076 diyesi
4628 ▁eyes
227798 ▁Egyes
13351 ▁yesterday
3276 yes
50873 ▁Reyes
154368 ▁menyes
218177 byes
65412 Yesterday
168681 ▁Yesus
84955 ▁Bayesian
21869 ▁YES
229064 bayes
214526 yesha
219433 ▁Noyes
80624 ▁dyes
124047 ▁Bayes
207027 Bayesian
7778 ▁yes
206236 ▁Keyes
197593 ▁yester
53071 Eyes
117162 ▁reyes
49846 ▁Hayes
37682 ▁polyester
124066 yesterday
33904 ▁Eyes
211223 ▁menyesuaikan
202425 ▁sayesinde
3553 Yes
54558 ▁Polyester
142981 ▁eyesight
17480 YES
54924 eyes
76725 ▁leyes
98768 yesi
6287 ▁Yes
172913 Polyester
113011 ▁eyeshadow
203066 Bayes
89792 ▁Yesterday
140879 ▁EYES


In [4]:
text = "I ate bread because I was feeling blue. What do you think?"
inputs = tokenizer(text, return_tensors="pt")
output = model.generate(inputs['input_ids'], max_new_tokens=25)
output = output[0].tolist()

print(text)
print(tokenizer.decode(output[inputs['input_ids'].shape[-1]:], 
                       skip_special_tokens=True, 
                       pad_token_id=tokenizer.eos_token_id))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


I ate bread because I was feeling blue. What do you think?


2024-11-19 15:53:51.910452: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 15:53:51.928354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-19 15:53:51.947700: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-19 15:53:51.953090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 15:53:51.966047: I tensorflow/core/platform/cpu_feature_guar



I ate bread because I was feeling blue. What do you think?

I ate bread because I was feeling blue.
